In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
OUTPUT

{

    "compound_name": "silver_bullet",
    "text_embedding": tensor([[...]]),  # text embedding，(1, 512)
    "images": [
        {
            "image_id": "image1.jpg",
            "image_embedding": tensor([[...]]),  #picture1 embedding，shape (1, 512)
        },
        {
            "image_id": "image2.jpg",
            "image_embedding": tensor([[...]]),  # picture2 embedding，shape is (1, 512)
        },
    ]

}


In [8]:
import os
import json
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

main_folder_path = "/content/drive/MyDrive/train"
json_file_path = "/content/drive/MyDrive/train_idiom_mean.json"

with open(json_file_path, "r", encoding="utf-8") as json_file:
    compound_definitions = json.load(json_file)

compound_data = []

for compound_name, compound_text in compound_definitions.items():
    compound_path = os.path.join(main_folder_path, compound_name)

    if not os.path.isdir(compound_path):
        print(f"Skipping compound: {compound_name} (folder does not exist)")
        continue

    print(f"Processing compound: {compound_name}")

    text_inputs = processor(text=[compound_text], return_tensors="pt", padding=True)
    with torch.no_grad():
        text_embedding = model.get_text_features(**text_inputs).squeeze().unsqueeze(0)  # (1, 512)

    images = []

    for filename in os.listdir(compound_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            image_path = os.path.join(compound_path, filename)

            try:
                image = Image.open(image_path).convert("RGB")
            except Exception as e:
                print(f"Error opening image {filename}: {e}")
                continue

            image_inputs = processor(images=image, return_tensors="pt", padding=True)
            with torch.no_grad():
                image_embedding = model.get_image_features(**image_inputs).squeeze().unsqueeze(0)  # (1, 512)

            images.append({
                "image_id": filename,
                "image_embedding": image_embedding
            })

    compound_data.append({
        "compound_name": compound_name,
        "text_embedding": text_embedding,
        "images": images
    })

output_file = "clip_train_idiom_embeddings.pt"
torch.save(compound_data, output_file)
print(f"Embeddings saved to {output_file}")

loaded_data = torch.load(output_file)
for compound in loaded_data:
    print(f"Compound: {compound['compound_name']}")
    print(f"Text Embedding Shape: {compound['text_embedding'].shape}")
    for image in compound["images"]:
        print(f"  Image ID: {image['image_id']}")
        print(f"  Image Embedding Shape: {image['image_embedding'].shape}")
    print("-" * 50)


output_file = "clip_train_idiom_embeddings.pt"
torch.save(compound_data, output_file)
print(f"Embeddings saved to {output_file}")

Processing compound: elbow grease
Processing compound: night owl
Processing compound: heart of gold
Processing compound: agony aunt
Processing compound: shrinking violet
Processing compound: banana republic
Processing compound: private eye
Processing compound: pipe dream
Processing compound: rocket science
Processing compound: nest egg
Processing compound: bull market
Processing compound: beached whale
Processing compound: lounge lizard
Processing compound: bear market
Processing compound: white hat
Processing compound: smoking gun
Processing compound: old flame
Processing compound: ivory tower
Processing compound: black sheep
Processing compound: gravy train
Processing compound: rat race
Processing compound: spring chicken
Processing compound: inner circle
Processing compound: bad apple
Processing compound: honey trap
Processing compound: open book
Processing compound: baby blues
Processing compound: brain surgery
Processing compound: red flag
Processing compound: white elephant
Proce

<ipython-input-8-1585265f0f08>:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(output_file)


Embeddings saved to clip_train_idiom_embeddings.pt


In [9]:
main_folder_path = "/content/drive/MyDrive/AdMIRe Subtask A Dev/dev"
json_file_path = "/content/drive/MyDrive/dev_mean.json"

with open(json_file_path, "r", encoding="utf-8") as json_file:
    compound_definitions = json.load(json_file)

compound_data = []

for compound_name, compound_text in compound_definitions.items():
    compound_path = os.path.join(main_folder_path, compound_name)

    if not os.path.isdir(compound_path):
        print(f"Skipping compound: {compound_name} (folder does not exist)")
        continue

    print(f"Processing compound: {compound_name}")

    text_inputs = processor(text=[compound_text], return_tensors="pt", padding=True)
    with torch.no_grad():
        text_embedding = model.get_text_features(**text_inputs).squeeze().unsqueeze(0)  # (1, 512)

    images = []

    for filename in os.listdir(compound_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            image_path = os.path.join(compound_path, filename)

            try:
                image = Image.open(image_path).convert("RGB")
            except Exception as e:
                print(f"Error opening image {filename}: {e}")
                continue

            image_inputs = processor(images=image, return_tensors="pt", padding=True)
            with torch.no_grad():
                image_embedding = model.get_image_features(**image_inputs).squeeze().unsqueeze(0)  # (1, 512)

            images.append({
                "image_id": filename,
                "image_embedding": image_embedding
            })

    compound_data.append({
        "compound_name": compound_name,
        "text_embedding": text_embedding,
        "images": images
    })

output_file = "clip_dev_embeddings.pt"
torch.save(compound_data, output_file)
print(f"Embeddings saved to {output_file}")

loaded_data = torch.load(output_file)
for compound in loaded_data:
    print(f"Compound: {compound['compound_name']}")
    print(f"Text Embedding Shape: {compound['text_embedding'].shape}")
    for image in compound["images"]:
        print(f"  Image ID: {image['image_id']}")
        print(f"  Image Embedding Shape: {image['image_embedding'].shape}")
    print("-" * 50)

Processing compound: monkey business
Processing compound: grass roots
Processing compound: marching orders
Processing compound: panda car
Processing compound: bread and butter
Processing compound: chocolate teapot
Skipping compound: pig's ear (folder does not exist)
Processing compound: best man
Processing compound: big cheese
Processing compound: eager beaver
Processing compound: hair of the dog
Processing compound: thin ice
Processing compound: snake in the grass
Processing compound: flea market
Processing compound: big fish
Embeddings saved to clip_dev_embeddings.pt
Compound: monkey business
Text Embedding Shape: torch.Size([1, 512])
  Image ID: 61570020623.png
  Image Embedding Shape: torch.Size([1, 512])
  Image ID: 94990180734.png
  Image Embedding Shape: torch.Size([1, 512])
  Image ID: 33778559524.png
  Image Embedding Shape: torch.Size([1, 512])
  Image ID: 04129294826.png
  Image Embedding Shape: torch.Size([1, 512])
  Image ID: 56875274126.png
  Image Embedding Shape: torch.

<ipython-input-9-f022996c8070>:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(output_file)


In [ ]:
main_folder_path = "/content/drive/MyDrive/train"
json_file_path = "/content/drive/MyDrive/train_literal_mean.json"

with open(json_file_path, "r", encoding="utf-8") as json_file:
    compound_definitions = json.load(json_file)

compound_data = []

for compound_name, compound_text in compound_definitions.items():
    compound_path = os.path.join(main_folder_path, compound_name)

    if not os.path.isdir(compound_path):
        print(f"Skipping compound: {compound_name} (folder does not exist)")
        continue

    print(f"Processing compound: {compound_name}")

    text_inputs = processor(text=[compound_text], return_tensors="pt", padding=True)
    with torch.no_grad():
        text_embedding = model.get_text_features(**text_inputs).squeeze().unsqueeze(0)  # (1, 512)

    images = []

    for filename in os.listdir(compound_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            image_path = os.path.join(compound_path, filename)

            try:
                image = Image.open(image_path).convert("RGB")
            except Exception as e:
                print(f"Error opening image {filename}: {e}")
                continue

            image_inputs = processor(images=image, return_tensors="pt", padding=True)
            with torch.no_grad():
                image_embedding = model.get_image_features(**image_inputs).squeeze().unsqueeze(0)  # (1, 512)

            images.append({
                "image_id": filename,
                "image_embedding": image_embedding
            })

    compound_data.append({
        "compound_name": compound_name,
        "text_embedding": text_embedding,
        "images": images
    })

output_file = "clip_train_literal_embeddings.pt"
torch.save(compound_data, output_file)
print(f"Embeddings saved to {output_file}")

loaded_data = torch.load(output_file)
for compound in loaded_data:
    print(f"Compound: {compound['compound_name']}")
    print(f"Text Embedding Shape: {compound['text_embedding'].shape}")
    for image in compound["images"]:
        print(f"  Image ID: {image['image_id']}")
        print(f"  Image Embedding Shape: {image['image_embedding'].shape}")
    print("-" * 50)

Processing compound: green fingers
Processing compound: ancient history
Skipping compound: devil's advocate (folder does not exist)
Processing compound: piece of cake
Processing compound: brass ring
Processing compound: apples and oranges
Processing compound: ghost town
Processing compound: flower child
Processing compound: copy cat
Processing compound: secret santa
